## Import Library

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_columns',100)
pd.set_option('Display.max_rows',100)

## Import Dataset

In [16]:
# Read csv file
path    = "../../final_assignment_public/Telecom_customer_churn.csv"
df      = pd.read_csv( path )

print( 'df:\n{} rows × {} columns\n'.format( df.shape[0], df.shape[1] ) )

df:
100000 rows × 100 columns



In [17]:
# Read csv file
path = '../csv/f_imp_core.csv'
df_feature = pd.read_csv( path, index_col=0 )

print( 'df:\n{} rows × {} columns\n'.format( df_feature.shape[0], df_feature.shape[1] ) )

df:
22 rows × 1 columns



## データの説明（日本語訳）
|  重要度  |  カラム名  |  内容  |
| ---- | ---- | ---- |
|  1  |  eqpdays  |  現在の端末の使用日数  |
|  2  |  months  |  契約してから何ヶ月か  |
|  3  |  change_mou  |  過去3か月の平均に対する月間使用時間の変化率  |
|  4  |  mou_Mean  |  毎月の平均使用分数  |
|  5  |  totmrc_Mean  |  毎月の請求額の平均  |
|  6  |  hnd_price  |  現在の携帯電話の価格  |
|  7  |  avgqty  |  顧客の生涯にわたる平均月間通話数  |
|  8(NG)  |  lor  |  居住期間  |
|  9(NG)  |  mou_cvce_Mean  |  完了した音声通話の平均使用時間  |
|  10 |  change_rev  |  過去3か月の平均に対する月間収益の変化率  |
|  11(NG) |  Customer_ID  |  NaN  |
|  12 |  totcalls  |  顧客の生涯にわたる通話の総数  |
|  13 |  avg3mou  |  過去3か月間の平均月間使用時間  |
|  14(cat) |  crclscod  |  クレジットクラスコード  |
|  15(cat) |  refurb_new  |  ハンドセット：再生品または新品  |
|  16(cat) |  hnd_webcap  |  ハンドセットのWeb機能  |
|  17(NG) |  avgrev  |  顧客の生涯にわたる平均月間収益  |
|  18(NG) |  ovrmou_Mean  |  使用の平均超過分数  |
|  19(NG) |  drop_vce_Mean  |  切断された（失敗した）音声通話の平均数  |
|  20(NG) |  phones  |  発行された携帯電話の数  |
|  21(NG) |  ovrrev_Mean  |  平均超過収入  |
|  22(NG) |  mouiwylisv_Mean  |  アウトバウンドワイヤレスからワイヤレスへの音声通話の平均使用時間  |

In [19]:
# 必要なカラムだけ取得
df_core = df[df_feature['feature']]
df_core['churn'] = df['churn']
df_core

,eqpdays,months,change_mou,mou_Mean,totmrc_Mean,hnd_price,avgqty,lor,mou_cvce_Mean,change_rev,Customer_ID,totcalls,avg3mou,crclscod,refurb_new,hnd_webcap,avgrev,ovrmou_Mean,drop_vce_Mean,phones,ovrrev_Mean,mouiwylisv_Mean,churn
0,361.0,61,-157.25,219.25,22.500,149.98999,32.69,15.0,90.643333,-18.9975,1000001,1652,272,A,N,WCMB,29.66,0.00,0.666667,2.0,0.0,0.000000,1
1,240.0,56,532.25,482.75,37.425,NaN,265.89,1.0,189.396667,50.9875,1000002,14654,305,EA,N,WC,51.53,22.75,8.333333,7.0,9.1,3.696667,0
2,1504.0,58,-4.25,10.25,16.990,29.98999,140.86,7.0,5.426667,0.0000,1000003,7903,12,C,N,NaN,34.54,0.00,0.333333,2.0,0.0,0.000000,1
3,1812.0,60,-1.50,7.50,38.000,29.98999,30.81,6.0,8.410000,0.0000,1000004,1502,8,B,N,NaN,40.45,0.00,0.000000,1.0,0.0,0.000000,0
4,434.0,57,38.50,570.50,71.980,149.98999,79.50,5.0,285.233333,0.0000,1000005,4485,558,A,N,WCMB,38.69,0.00,9.666667,6.0,0.0,0.366667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,773.0,25,NaN,NaN,NaN,29.98999,178.62,15.0,94.866667,NaN,1099996,3771,410,B,N,WC,74.34,NaN,2.000000,1.0,NaN,0.000000,1
99996,835.0,27,NaN,NaN,NaN,29.98999,181.45,9.0,100.760000,NaN,1099997,3675,382,CY,N,WC,72.59,NaN,3.000000,1.0,NaN,0.000000,0
99997,433.0,31,NaN,NaN,NaN,199.98999,65.16,NaN,0.000000,NaN,1099998,1271,662,DA,N,WCMB,55.58,NaN,0.000000,2.0,NaN,0.000000,0
99998,75.0,25,NaN,NaN,NaN,149.98999,472.43,NaN,564.233333,NaN,1099999,10082,1031,EA,N,WCMB,123.49,NaN,18.666667,7.0,NaN,32.883333,0


## Data Preprocessing

### Categorical data

In [42]:
# カテゴリカル変数を表示
x_cat = df_core[df_core.select_dtypes( include=object ).columns].columns

# カテゴリカル変数を落とす(理解不能なため)
df_core = df_core.drop( columns=x_cat )
df_core

,eqpdays,months,change_mou,mou_Mean,totmrc_Mean,hnd_price,avgqty,lor,mou_cvce_Mean,change_rev,Customer_ID,totcalls,avg3mou,avgrev,ovrmou_Mean,drop_vce_Mean,phones,ovrrev_Mean,mouiwylisv_Mean,churn
0,361.0,61,-157.25,219.25,22.500,149.98999,32.69,15.0,90.643333,-18.9975,1000001,1652,272,29.66,0.00,0.666667,2.0,0.0,0.000000,1
1,240.0,56,532.25,482.75,37.425,NaN,265.89,1.0,189.396667,50.9875,1000002,14654,305,51.53,22.75,8.333333,7.0,9.1,3.696667,0
2,1504.0,58,-4.25,10.25,16.990,29.98999,140.86,7.0,5.426667,0.0000,1000003,7903,12,34.54,0.00,0.333333,2.0,0.0,0.000000,1
3,1812.0,60,-1.50,7.50,38.000,29.98999,30.81,6.0,8.410000,0.0000,1000004,1502,8,40.45,0.00,0.000000,1.0,0.0,0.000000,0
4,434.0,57,38.50,570.50,71.980,149.98999,79.50,5.0,285.233333,0.0000,1000005,4485,558,38.69,0.00,9.666667,6.0,0.0,0.366667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,773.0,25,NaN,NaN,NaN,29.98999,178.62,15.0,94.866667,NaN,1099996,3771,410,74.34,NaN,2.000000,1.0,NaN,0.000000,1
99996,835.0,27,NaN,NaN,NaN,29.98999,181.45,9.0,100.760000,NaN,1099997,3675,382,72.59,NaN,3.000000,1.0,NaN,0.000000,0
99997,433.0,31,NaN,NaN,NaN,199.98999,65.16,NaN,0.000000,NaN,1099998,1271,662,55.58,NaN,0.000000,2.0,NaN,0.000000,0
99998,75.0,25,NaN,NaN,NaN,149.98999,472.43,NaN,564.233333,NaN,1099999,10082,1031,123.49,NaN,18.666667,7.0,NaN,32.883333,0


### Numerical data

In [44]:
def check_null( _df ):
    print( _df.isnull().sum() )

In [46]:
# 欠損が多いカラムを落とす
df_core = df_core.drop( columns='lor' )

In [50]:
# その他不要なカラムを落とす
df_core = df_core.drop( columns=['Customer_ID', 'mouiwylisv_Mean'] )

## EDA

In [79]:
# 解約者と非解約者で，明らかに差がありそうなデータを探す
for column_name in df_core.columns:
    print( '{}\n'.format( df_core.groupby('churn')[column_name].mean() ) )

churn
0    363.280925
1    421.089524
Name: eqpdays, dtype: float64

churn
0    18.633074
1    19.038457
Name: months, dtype: float64

churn
0    -5.344265
1   -22.759003
Name: change_mou, dtype: float64

churn
0    543.206895
1    483.306417
Name: mou_Mean, dtype: float64

churn
0    47.782378
1    44.543091
Name: totmrc_Mean, dtype: float64

churn
0    108.129344
1     95.539523
Name: hnd_price, dtype: float64

churn
0    177.505869
1    169.533305
Name: avgqty, dtype: float64

churn
0   -1.410195
1   -0.621263
Name: change_rev, dtype: float64

churn
0    2936.439609
1    2816.796174
Name: totcalls, dtype: float64

churn
0    545.850073
1    492.969109
Name: avg3mou, dtype: float64

churn
0    0
1    1
Name: churn, dtype: int64



In [78]:
# 解約者と非解約者で，明らかに差がありそうなデータを探す
for column_name in df_core.columns:
    print( '{}\n'.format( df_core.groupby('churn')[column_name].std() ) )

churn
0    249.986138
1    259.705607
Name: eqpdays, dtype: float64

churn
0    9.975607
1    9.314720
Name: months, dtype: float64

churn
0    250.239558
1    300.083099
Name: change_mou, dtype: float64

churn
0    540.310876
1    507.474688
Name: mou_Mean, dtype: float64

churn
0    24.263099
1    22.836931
Name: totmrc_Mean, dtype: float64

churn
0    61.123188
1    60.228330
Name: hnd_price, dtype: float64

churn
0    175.447096
1    159.595591
Name: avgqty, dtype: float64

churn
0    35.996984
1    61.732442
Name: change_rev, dtype: float64

churn
0    4086.929791
1    3462.678410
Name: totcalls, dtype: float64

churn
0    549.270722
1    515.876242
Name: avg3mou, dtype: float64

churn
0    0.0
1    0.0
Name: churn, dtype: float64



In [70]:
# 改めて，不要なカラム(groupbyの結果，解約者と非解約者で差がないデータ)を落とす
df_core = df_core.drop( columns=['mou_cvce_Mean', 'drop_vce_Mean', 'ovrmou_Mean', 'phones', 'ovrrev_Mean', 'avgrev'] )

KeyError: "['mou_cvce_Mean' 'drop_vce_Mean' 'ovrmou_Mean' 'phones' 'ovrrev_Mean'] not found in axis"

### 残ったデータ
|  重要度  |  カラム名  |  内容  |  分類  |
| ---- | ---- | ---- | ---- |
|  1  |  eqpdays  |  現在の端末の使用日数  |  使用日数  |
|  2  |  months  |  契約してから何ヶ月か  |  使用月数  |
|  3  |  change_mou  |  過去3か月の月間平均使用時間の変化率  |  使用時間  |
|  4  |  mou_Mean  |  毎月の平均使用分数  |  使用時間  |
|  5  |  totmrc_Mean  |  毎月の請求額の平均  |  金額  |
|  6  |  hnd_price  |  現在の携帯電話の価格  |  金額  |
|  7  |  avgqty  |  顧客の生涯にわたる平均月間通話数  |  通話数  |
|  8 |  change_rev  |  過去3か月の平均に対する月間収益の変化率  |  収益  |
|  9 |  totcalls  |  顧客の生涯にわたる通話の総数  |  通話数  |
|  10 |  avg3mou  |  過去3か月間の平均月間使用時間  |  使用時間  |

In [ ]:
def plot():
    plt.rcParams["figure.figsize"] = (10, 10)
    plt.title( 'Cumulative Feature Importance', fontsize=24 )

    x = df_importance['feature']
    y = df_importance['cum_imp']
    plt.bar( x, y )
    # plt.hlines( 0.8, 0, 99, "red", linestyles='dashed')
    plt.xticks( range( df_importance['feature'].size ), x, rotation=90, fontsize=10 )
    plt.yticks( [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=18 )
    plt.ylabel( 'Normalized Feature Importance', fontsize=18 )
    plt.show()

In [82]:
df_core.groupby('churn')['change_mou'].mean()

churn
0    -5.344265
1   -22.759003
Name: change_mou, dtype: float64

In [25]:
def plot_hist(x):
    plt.rcParams["figure.figsize"] = (20, 4)
    ax = sns.countplot( x=x, data=df_core )
    plt.xlabel( str(x) )
    plt.title( 'Histogram of ' + str(x) )
    plt.xticks( rotation=70 )
    # plt.xticks()
    plt.show()

In [26]:
x_call = df_core.columns

Index(['eqpdays', 'months', 'change_mou', 'mou_Mean', 'totmrc_Mean',
       'hnd_price', 'avgqty', 'lor', 'mou_cvce_Mean', 'change_rev',
       'Customer_ID', 'totcalls', 'avg3mou', 'crclscod', 'refurb_new',
       'hnd_webcap', 'avgrev', 'ovrmou_Mean', 'drop_vce_Mean', 'phones',
       'ovrrev_Mean', 'mouiwylisv_Mean', 'churn'],
      dtype='object')

In [ ]:
for x in x_call:
    plot_hist( x )

### Numerical data

In [ ]:
# 数値変数を確認
numerical = df[df.select_dtypes( exclude=object ).columns]
x_num = numerical.columns
numerical.head()

In [ ]:
# 欠損値を0で埋める
numerical = numerical.fillna(0)
check_null( numerical )

In [ ]:
# 反映
df[x_num] = numerical

### Categorical data encoding

In [ ]:
# カテゴリカル変数を確認
categorical = df[df.select_dtypes( include=object ).columns]
x_cat = categorical.columns
categorical.head()

In [ ]:
def plot_hist(x):
    plt.rcParams["figure.figsize"] = (20, 4)
    ax = sns.countplot( x=x, data=df )
    plt.xlabel( str(x) )
    plt.title( 'Histogram of ' + str(x) )
    plt.xticks( rotation=70 )
    # plt.xticks()
    plt.show()

In [ ]:
for x in x_cat:
    plot_hist( x )

In [ ]:
categorical = categorical.dropna( axis=0 )
check_null( categorical )

In [ ]:
# 反映
df[x_cat] = categorical

In [ ]:
df = df.dropna( axis=0 )
check_null( df )

In [ ]:
# For Categorical data, converted to numerical using Label Encoder. 
# By this method, all categorical data are sorted by alphabetically.
for x in x_cat:
    df[x] = LabelEncoder().fit_transform( df[x] )

In [ ]:
col_num = df[df.select_dtypes( exclude=object ).columns].columns
df = pd.DataFrame( df[col_num] )
df

## Train Test Split

In [ ]:
# y = df['churn']
# X_train, X_test, y_train, y_test = train_test_split( df, y, random_state=1 )

# churnを目的変数に設定
y_train = df['churn']

# 目的変数のみ落とす
X_train = df.drop( columns=['churn'] )

In [ ]:
# LightGBM parameters

# lgbm_params = {
#         'reg_lambda': 0.06875461198761648, 
#         'reg_alpha': 2.2293651627955215, 
#         'colsample_bytree': 0.7208495854217002, 
#         'subsample': 0.781401201566485, 
#         'max_depth': 8, 
#         'min_child_weight': 3.109569481392587
# }

# lgbm_params = {
#         'boosting_type': 'gbdt',
#         'objective': 'regression',
#         'metric': 'rmse',
#         'max_depth': 5,
#         'min_data_in_leaf': 20,
#         'min_child_weight': 3,
#         'feature_fraction': 1.0,
#         'min_data_in_leaf': 20,
#         'num_iterations': 100,
#         'seed': 0,
#         'verbose': -1,
#         'reg_lambda': 0.07, 
#         'reg_alpha': 2.23, 
#         'colsample_bytree': 0.72, 
#         'subsample': 0.78
# }

# LightGBM model
model_lgbm = LGBMClassifier(
        boosting_type='gbdt', 
        class_weight='balanced',
        reg_lambda=1.1564659040946654, 
        reg_alpha=9.90877329623665,
        colsample_bytree=0.5034991685866442, 
        subsample=0.6055998601661783, 
        max_depth=5,
        min_child_weight=39.72586351155486, 
        learning_rate=0.0732489659779158,
        num_threads=1 )

In [ ]:
# model_lgbm.fit( train_df[x_calls], train_df['TARGET'] )
model_lgbm.fit( X_train, y_train )

# clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

In [ ]:
lgbm.plot_importance( model_lgbm, importance_type="gain", max_num_features=100, figsize=(10, 20) )

In [ ]:
f_name = model_lgbm.feature_name_

# f_importance = model_lgbm.feature_importances_
f_importance = model_lgbm.booster_.feature_importance( importance_type='gain' )
f_importance = f_importance / np.sum( f_importance )

df_importance = pd.DataFrame( {'feature':f_name, 'importance':f_importance} )
df_importance = df_importance.sort_values('importance', ascending=False)

# インデックスを振り直す
df_importance = df_importance.reset_index()

display( df_importance )

In [ ]:
# 寄与率が低いカラムをリストアップ
f_mean = df_importance['importance'].mean()
f_sd = df_importance['importance'].std()
print( 'mean: {}'.format( f_mean ) )
print( 'sd: {}'.format( f_sd ) )

print( 'mean +   sd: {}'.format( f_mean +   f_sd ) )
print( 'mean + 2*sd: {}'.format( f_mean + 2*f_sd ) )
print( 'mean + 3*sd: {}'.format( f_mean + 3*f_sd ) )

In [ ]:
# 累積寄与率
# この○個のデータが，結果の80%を占めている．
tmp_sum = 0
cum_importance = []
for i in range( 98 ):
    tmp_sum += df_importance['importance'][i]

    if tmp_sum < 0.8:
        f_th = i+1
        print( '{}. {}:\n   {}\n'.format( i+1, df_importance['feature'][i], tmp_sum ) )
        cum_importance.append( tmp_sum )

In [ ]:
df_importance_core = df_importance[:f_th]
df_importance_core['cum_importance'] = cum_importance

display( df_importance_core )

In [ ]:
def plot_cum_importance( _np_array ):
    

In [ ]:
plot_cum_importance( df_importance_core['cum_importance'] )

In [ ]:
display( df_importance_core['feature'] )

In [ ]:
df_importance_core['feature'].to_csv( './f_importance_core.csv' )